### 함수화


`05-Assistant-API.ipynb`에서 assisant api code는  흩어져 있음.

따라서, 내용 묶어서 함수화 코드 실습 시작


In [2]:
from openai import OpenAI
import json 

def show_json(obj):
    # obj의 모델을 JSON 형태로 변환한 후 출력합니다.
    display(json.loads(obj.model_dump_json()))
    
# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = OpenAI()

# 수학 과외 선생님 역할을 하는 챗봇을 생성합니다.
# 이 챗봇은 간단한 문장이나 한 문장으로 질문에 답변합니다.
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4o-mini",
)
# 생성된 챗봇의 정보를 JSON 형태로 출력합니다.
show_json(assistant)

{'id': 'asst_TbKNyLDSJiXr7T3jAPyNItPR',
 'created_at': 1722441833,
 'description': None,
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4o-mini',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'top_p': 1.0}

In [3]:
import time
from openai import OpenAI


# 이전에 설정한 Assistant ID 를 기입합니다.
ASSISTANT_ID = assistant.id

# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = OpenAI()


def submit_message(assistant_id, thread, user_message):
    # 사용자 입력 메시지를 스레드에 추가합니다.
    client.beta.threads.messages.create(
        # Thread ID가 필요합니다.
        # 사용자 입력 메시지 이므로 role은 "user"로 설정합니다.
        # 사용자 입력 메시지를 content에 지정합니다.
        thread_id=thread.id,
        role="user",
        content=user_message,
    )
    # 스레드에 메시지가 입력이 완료되었다면,
    # Assistant ID와 Thread ID를 사용하여 실행을 준비합니다.
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )
    return run


def wait_on_run(run, thread):
    # 주어진 실행(run)이 완료될 때까지 대기합니다.
    # status 가 "queued" 또는 "in_progress" 인 경우에는 계속 polling 하며 대기합니다.
    while run.status == "queued" or run.status == "in_progress":
        # run.status 를 업데이트합니다.
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        # API 요청 사이에 잠깐의 대기 시간을 두어 서버 부하를 줄입니다.
        time.sleep(0.5)
    return run


def get_response(thread):
    # 스레드에서 메시지 목록을 가져옵니다.
    # 메시지를 오름차순으로 정렬할 수 있습니다. order="asc"로 지정합니다.
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

재사용할 수 있는 `create_thread_and_run` 함수를 정의했습니다(사실상 우리 API의 [`client.beta.threads.create_and_run`](https://platform.openai.com/docs/api-reference/runs/createThreadAndRun) 복합 함수와 거의 동일합니다)

`create_thread_and_run` 함수는 새로운 스레드를 생성하고 실행하기 위한 준비단계(status 가 `queued` 된 상태) 까지 진행합니다.

이러한 API 호출이 모두 비동기 작업인 점을 알아두시면 좋습니다.


In [4]:
# 새로운 스레드를 생성하고 메시지를 제출하는 함수를 정의합니다.
def create_thread_and_run(user_input):
    # 사용자 입력을 받아 새로운 스레드를 생성하고, Assistant 에게 메시지를 제출합니다.
    thread = client.beta.threads.create()
    run = submit_message(ASSISTANT_ID, thread, user_input)
    return thread, run

비동기로 `queued` 상태인 `Run` 을 생성했습니다.

아직 실행이 시작된 것은 아니라는 점을 주의해 주세요.


In [5]:
# 동시에 여러 요청을 처리하기 위해 스레드를 생성합니다.
thread1, run1 = create_thread_and_run(
    "다음 방정식을 풀고 싶습니다. `3x + 11 = 14`. 좀 도와주실 수 있나요?"
)
thread2, run2 = create_thread_and_run("선형대수학에 대해 간략히 설명해 주실 수 있나요?")
thread3, run3 = create_thread_and_run(
    "수학에 정말 소질이 없는 것 같아. 어떻게 하면 수학을 잘할 수 있을까요? 조언좀 해줘요."
)

모든 실행이 진행되고 나면, 각각을 기다린 후 응답을 받을 수 있습니다.


In [6]:
import time


# 메시지 출력용 함수
def print_message(response):
    for res in response:
        print(f"[{res.role.upper()}]\n{res.content[0].text.value}\n")
    print("---" * 20)


# 반복문에서 대기하는 함수


def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# 첫 번째 실행을 위해 대기
run1 = wait_on_run(run1, thread1)
print_message(get_response(thread1))

# 두 번째 실행을 위해 대기
run2 = wait_on_run(run2, thread2)
print_message(get_response(thread2))

# 세 번째 실행을 위해 대기
run3 = wait_on_run(run3, thread3)
# 세 번째 스레드를 마치면 감사 인사를 전하고 종료합니다 :)
run4 = submit_message(ASSISTANT_ID, thread3, "도와주셔서 감사합니다!")
run4 = wait_on_run(run4, thread3)
print_message(get_response(thread3))

[USER]
다음 방정식을 풀고 싶습니다. `3x + 11 = 14`. 좀 도와주실 수 있나요?

[ASSISTANT]
네, 방정식을 풀면 \( x = 1 \)입니다.

------------------------------------------------------------
[USER]
선형대수학에 대해 간략히 설명해 주실 수 있나요?

[ASSISTANT]
선형대수학은 벡터 공간과 선형 변환을 다루는 수학의 한 분야로, 행렬, 벡터, 시스템의 해 등을 연구합니다.

------------------------------------------------------------
[USER]
수학에 정말 소질이 없는 것 같아. 어떻게 하면 수학을 잘할 수 있을까요? 조언좀 해줘요.

[ASSISTANT]
수학을 잘하기 위해서는 기초 개념을 확실히 이해하고, 꾸준히 연습하며, 문제를 푸는 습관을 기르는 것이 중요합니다.

[USER]
도와주셔서 감사합니다!

[ASSISTANT]
천만에요! 언제든지 질문해 주세요!

------------------------------------------------------------


### 전체코드(템플릿 코드)


API KEY를 설정하고, helper 함수를 정의합니다.


In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
import os
import json

api_key = os.environ.get("OPENAI_API_KEY")


def show_json(obj):
    # obj의 모델을 JSON 형태로 변환한 후 출력합니다.
    display(json.loads(obj.model_dump_json()))

1. Assistant 생성
   1. [Assistants Playground](https://platform.openai.com/playground) 에서 이미 Assistant 를 생성한 경우
   2. Assistant 를 생성하지 않은 경우


In [9]:
# 1-1. Assistant ID를 불러옵니다(Playground에서 생성한 Assistant ID)
ASSISTANT_ID = "asst_V8s4Ku4Eiid5QC9WABlwDsYs"

In [11]:
# 1-2. Assistant 를 생성합니다.
from openai import OpenAI

# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = OpenAI(api_key=api_key)

# Assistant 를 생성합니다.
assistant = client.beta.assistants.create(
    name="Math Tutor",  # 챗봇의 이름을 지정합니다.
    # 챗봇의 역할을 설명합니다.
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4o-mini",  # 사용할 모델을 지정합니다.
)

# 생성된 챗봇의 정보를 JSON 형태로 출력합니다.
show_json(assistant)
ASSISTANT_ID = assistant.id

{'id': 'asst_YBFC5ROtYmKYjsH7CI68V6Tz',
 'created_at': 1722442008,
 'description': None,
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4o-mini',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'top_p': 1.0}

2. 스레드(Thread) 생성하기
   1. 스레드를 이미 생성한 경우
   2. 스레드를 새롭게 생성하는 경우


In [12]:
# 2-1. 스레드를 이미 생성한 경우
THREAD_ID = "thread_6We5fHvb5NBuacPfZYkqUWlO"

In [13]:
# 2-2. 스레드를 새롭게 생성합니다.
def create_new_thread():
    # 새로운 스레드를 생성합니다.
    thread = client.beta.threads.create()
    return thread


thread = create_new_thread()
# 새로운 스레드를 생성합니다.
show_json(thread)
# 새롭게 생성한 스레드 ID를 저장합니다.
THREAD_ID = thread.id

{'id': 'thread_QNIvnBlhUvHXoKXWOdD2q0Gy',
 'created_at': 1722442016,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

3. 스레드에 메시지 생성
   1. 스레드에 새로운 메시지를 추가 합니다.
   2. 스레드를 실행(run) 합니다.
   3. 스레드의 상태를 확인합니다.(대기중, 작업중, 완료, etc)
   4. 스레드에서 최신 메시지를 조회한 뒤 결과를 확인합니다.


In [14]:
import time


# 반복문에서 대기하는 함수
def wait_on_run(run, thread_id):
    while run.status == "queued" or run.status == "in_progress":
        # 3-3. 실행 상태를 최신 정보로 업데이트합니다.
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


def submit_message(assistant_id, thread_id, user_message):
    # 3-1. 스레드에 종속된 메시지를 '추가' 합니다.
    client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=user_message
    )
    # 3-2. 스레드를 실행합니다.
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
    )
    return run


def get_response(thread_id):
    # 3-4. 스레드에 종속된 메시지를 '조회' 합니다.
    return client.beta.threads.messages.list(thread_id=thread_id, order="asc")


def print_message(response):
    for res in response:
        print(f"[{res.role.upper()}]\n{res.content[0].text.value}\n")


def ask(assistant_id, thread_id, user_message):
    run = submit_message(
        assistant_id,
        thread_id,
        user_message,
    )
    # 실행이 완료될 때까지 대기합니다.
    run = wait_on_run(run, thread_id)
    print_message(get_response(thread_id).data[-2:])
    return run

In [ ]:
# thread_id = "기존 스레드 ID를 입력해 주세요"
thread_id = create_new_thread().id  # 새로운 스레드를 생성합니다.
run = ask(ASSISTANT_ID, thread_id, "I need to solve `1 + 20`. Can you help me?")

[USER]
I need to solve `1 + 20`. Can you help me?

[ASSISTANT]
Yes, \(1 + 20 = 21\).



In [ ]:
# 전체 대화내용 출력
print_message(get_response(thread_id).data[:])

[USER]
I need to solve `1 + 20`. Can you help me?

[ASSISTANT]
Yes, \(1 + 20 = 21\).



## 전체를 class 로 만들어보기
- 사용자입장에서 보면 
- assistant를 지정하고
- 질문을 하고
- 답변을 받으면 됨

In [17]:
from openai import OpenAI
import os
import time

class MathTutor:
    def __init__(self):
        # OpenAI 클라이언트 초기화
        self.client = OpenAI()
        
        # Assistant 생성
        self.assistant = self.client.beta.assistants.create(
            name="Math Tutor",
            instructions="You are a helpful math tutor. Your task is to assist students with mathematical problems, providing step-by-step explanations and solutions.",
            model="gpt-4o-mini"
        )
        
        # Thread 생성
        self.thread = self.client.beta.threads.create()

    def assistant_math_tutor(self, input_message):
        # 메시지 추가
        self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role="user",
            content=input_message
        )

        # Run 생성 및 완료 대기
        run = self.client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id
        )

        while True:
            run_status = self.client.beta.threads.runs.retrieve(
                thread_id=self.thread.id,
                run_id=run.id
            )
            if run_status.status == 'completed':
                break
            time.sleep(1)

        # 응답 가져오기
        messages = self.client.beta.threads.messages.list(thread_id=self.thread.id)
        assistant_response = messages.data[0].content[0].text.value

        return assistant_response

In [18]:
tutor = MathTutor()

# 첫 번째 질문
question1 = "y = 6x**2 + 2x - 5의 해를 구해주세요."
answer1 = tutor.assistant_math_tutor(question1)
print("Question 1:", question1)
print("Answer 1:", answer1)
    


Question 1: y = 6x**2 + 2x - 5의 해를 구해주세요.
Answer 1: 주어진 방정식 \( y = 6x^2 + 2x - 5 \)의 해를 찾기 위해서는 이 방정식을 \( y = 0 \)으로 두고, 즉 \( 6x^2 + 2x - 5 = 0 \)의 형태로 변형해 근을 찾아야 합니다. 이차 방정식의 해를 구하기 위해 근의 공식을 사용할 수 있습니다.

이차 방정식의 일반형은 다음과 같습니다:
\[
ax^2 + bx + c = 0
\]
여기서 \( a = 6 \), \( b = 2 \), \( c = -5 \)입니다.

근의 공식은 다음과 같습니다:
\[
x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
\]

이제 각 값을 대입하여 계산해보겠습니다.

1. **\( b^2 - 4ac \) 계산하기**
   \[
   b^2 = 2^2 = 4
   \]
   \[
   4ac = 4 \cdot 6 \cdot (-5) = -120
   \]
   \[
   b^2 - 4ac = 4 - (-120) = 4 + 120 = 124
   \]

2. **근의 공식에 대입하기**
   \[
   x = \frac{-2 \pm \sqrt{124}}{2 \cdot 6}
   \]
   \[
   = \frac{-2 \pm \sqrt{124}}{12}
   \]
   \(\sqrt{124}\)는 간단히 할 수 있습니다:
   \[
   \sqrt{124} = \sqrt{4 \times 31} = 2\sqrt{31}
   \]

   따라서 식은 다음과 같습니다:
   \[
   x = \frac{-2 \pm 2\sqrt{31}}{12}
   \]
   \[
   = \frac{-1 \pm \sqrt{31}}{6}
   \]

3. **결과**
   두 해는 다음과 같습니다:
\[
x_1 = \frac{-1 + \sqrt{31}}{6}, \quad x_2 = \frac{-1 - \sqrt{31}}{6}
\]

따라서 주어진 이차 방정식 \( 6x

In [ ]:
# 두 번째 질문
question2 = "sin(x) + cos(x) = 1의 해를 구해주세요."
answer2 = tutor.assistant_math_tutor(question2)
print("\nQuestion 2:", question2)
print("Answer 2:", answer2)


Question 2: sin(x) + cos(x) = 1의 해를 구해주세요.
Answer 2: 주어진 삼각방정식 sin(x) + cos(x) = 1의 해를 구하기 위해 방정식의 삼각함수 특성을 이용하여 해결할 수 있습니다.

먼저, 삼각함수의 합 차 공식을 적용해 sin(x) + cos(x)를 단일 삼각함수로 표현할 수 있습니다.

sin(x) + cos(x) = sqrt(2) * sin(x + pi/4)

따라서, 주어진 방정식은 다음과 같이 변환됩니다.

sqrt(2) * sin(x + pi/4) = 1

이제 삼각함수의 정의에 따라 sin(x) = 1 / sqrt(2)이 되는 경우를 생각할 수 있습니다.

sin(x) = 1 / sqrt(2)
x = π/4 + 2πn, 5π/4 + 2πn

따라서, 방정식 sin(x) + cos(x) = 1의 해는 x = π/4 + 2πn 또는 x = 5π/4 + 2πn (여기서 n은 정수) 입니다.
